# **Assignment 4: some database operations**
- **you will learn:** using joins, aggregation, groupby in Pandas
- **task:**  See section 4.2 below
- **deadline:** 17.11.2025
- [Pandas documentation](https://pandas.pydata.org/docs/index.html)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.

In [1]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

NumPy version: 2.3.3
Pandas version: 2.3.2


In [2]:
# -------------------------------
# 1) Build synthetic datasets
# -------------------------------
# sensor readouts (long-format)
np.random.seed(42)
N_SENSORS = 6
sensors = [f'S{i+1}' for i in range(N_SENSORS)]

#print(sensors)

start = pd.Timestamp('2025-11-01 00:00')
periods = 24 * 6 # six measurements per hour for 24 hours
freq = '10min' # 10-minute frequency
idx = pd.date_range(start=start, periods=periods, freq=freq)

#print(idx)


rows = []
for sensor in sensors:
    base_temp = 15 + 5 * np.random.rand() # baseline temp different per sensor
    base_hum = 40 + 20 * np.random.rand()
    noise_t = np.random.normal(0, 1.5, size=len(idx))
    noise_h = np.random.normal(0, 5.0, size=len(idx))
    # add a diurnal temperature variation
    hours = (idx.hour + idx.minute/60.0)
    diurnal = 6 * np.sin(2 * np.pi * (hours - 6) / 24)
    #print(diurnal[24*3:24*5])
    
    temps = base_temp + diurnal + noise_t
    hums = np.clip(base_hum - 0.5 * diurnal + noise_h, 0, 100)
    
    for ts, t, h in zip(idx, temps, hums):
        rows.append({'sensor_id': sensor, 'timestamp': ts, 'temperature': t, 'humidity': h})

sensor_df = pd.DataFrame(rows)


# add some missing values intentionally and some outliers
sensor_df.loc[sensor_df.sample(frac=0.01, random_state=1).index, 'temperature'] = np.nan
sensor_df.loc[sensor_df.sample(frac=0.005, random_state=2).index, 'humidity'] = np.nan
# inject outliers
outlier_idx = sensor_df.sample(frac=0.002, random_state=3).index
sensor_df.loc[outlier_idx, 'temperature'] += np.random.choice([20, -15], size=len(outlier_idx))


print('--- sensor_df (head) ---')
print(sensor_df.head())

# sensor metadata (wide)
meta = pd.DataFrame({
'sensor_id': sensors,
'location': ['Room A', 'Room B', 'Room C', 'Room A', 'Room D', 'Room B'],
'type': ['thermo-hygro', 'thermo-hygro', 'thermo', 'thermo-hygro', 'hygro', 'thermo']
})
print(meta)

# events table (sparse)
events = pd.DataFrame([
{'sensor_id': 'S1', 'timestamp': start + pd.Timedelta(hours=5), 'event': 'maintenance'},
{'sensor_id': 'S3', 'timestamp': start + pd.Timedelta(hours=8, minutes=20), 'event': 'calibration'},
{'sensor_id': 'S2', 'timestamp': start + pd.Timedelta(hours=18), 'event': 'spike_detected'}
])

--- sensor_df (head) ---
  sensor_id           timestamp  temperature   humidity
0        S1 2025-11-01 00:00:00    11.844233  55.829533
1        S1 2025-11-01 00:10:00    13.162956  55.409148
2        S1 2025-11-01 00:20:00    10.544302  64.612578
3        S1 2025-11-01 00:30:00          NaN  63.473544
4        S1 2025-11-01 00:40:00    13.332673  63.221174
  sensor_id location          type
0        S1   Room A  thermo-hygro
1        S2   Room B  thermo-hygro
2        S3   Room C        thermo
3        S4   Room A  thermo-hygro
4        S5   Room D         hygro
5        S6   Room B        thermo


In [3]:
# -------------------------------
# 2) MultiIndex: set index as (sensor_id, timestamp)
# -------------------------------
sensor_mi = sensor_df.set_index(['sensor_id', 'timestamp']).sort_index()
print('\n--- sensor_mi (index sample) ---')
print(sensor_mi.head())

# show selecting using MultiIndex
print('\nSelect all readings for S2 between two times:')
print(sensor_mi.loc['S2'].between_time('06:00', '09:00').head())


--- sensor_mi (index sample) ---
                               temperature   humidity
sensor_id timestamp                                  
S1        2025-11-01 00:00:00    11.844233  55.829533
          2025-11-01 00:10:00    13.162956  55.409148
          2025-11-01 00:20:00    10.544302  64.612578
          2025-11-01 00:30:00          NaN  63.473544
          2025-11-01 00:40:00    13.332673  63.221174

Select all readings for S2 between two times:
                     temperature   humidity
timestamp                                  
2025-11-01 06:00:00    16.292123  38.803557
2025-11-01 06:10:00    16.560178  53.083026
2025-11-01 06:20:00    15.860508  54.342884
2025-11-01 06:30:00    14.793219  44.742754
2025-11-01 06:40:00    16.511539  48.741353


In [4]:
# -------------------------------
# 3) Multi-level columns example: create aggregated summary with MultiIndex columns
# -------------------------------
agg_funcs = {
'temperature': ['mean', 'std', 'min', 'max'],
'humidity': ['mean', 'std']
}
summary_by_sensor = sensor_df.groupby('sensor_id').agg(agg_funcs)
# flatten/unflatten demonstration: keep as MultiIndex columns
print('\n--- summary_by_sensor (MultiIndex columns) ---')
print(summary_by_sensor.head())

# rename columns to readable MultiIndex (level names)
summary_by_sensor.columns.names = ['measurement', 'stat']
print('\nColumns levels:', summary_by_sensor.columns.names)
print(summary_by_sensor)

# access a specific subtable using xs
print('\nTemperature means:')
print(summary_by_sensor.xs('temperature', axis=1 )[['mean', 'std']])


--- summary_by_sensor (MultiIndex columns) ---
          temperature                                  humidity          
                 mean       std       min        max       mean       std
sensor_id                                                                
S1          16.718369  4.511676  8.552629  25.213955  59.259889  5.560005
S2          15.427998  4.497470  2.122613  24.023564  45.599224  5.493384
S3          15.658394  4.421943  7.273332  25.556400  56.385015  5.146519
S4          15.668049  4.238744  7.365955  24.204889  48.812680  5.493305
S5          17.801163  4.889507  9.133021  38.594542  49.913718  5.367881

Columns levels: ['measurement', 'stat']
measurement temperature                                  humidity          
stat               mean       std       min        max       mean       std
sensor_id                                                                  
S1            16.718369  4.511676  8.552629  25.213955  59.259889  5.560005
S2            1

In [5]:
summary_by_sensor.columns = [
    f"{measurement}_{stat}"
    for measurement, stat in summary_by_sensor.columns
]

print('\n--- summary_by_sensor (flat columns) ---')
print(summary_by_sensor.head())


--- summary_by_sensor (flat columns) ---
           temperature_mean  temperature_std  temperature_min  \
sensor_id                                                       
S1                16.718369         4.511676         8.552629   
S2                15.427998         4.497470         2.122613   
S3                15.658394         4.421943         7.273332   
S4                15.668049         4.238744         7.365955   
S5                17.801163         4.889507         9.133021   

           temperature_max  humidity_mean  humidity_std  
sensor_id                                                
S1               25.213955      59.259889      5.560005  
S2               24.023564      45.599224      5.493384  
S3               25.556400      56.385015      5.146519  
S4               24.204889      48.812680      5.493305  
S5               38.594542      49.913718      5.367881  


In [6]:
# -------------------------------
# 4) Advanced groupby with named aggregation and custom functions
# -------------------------------
# We'll compute per-sensor and per-location aggregations using named aggregations
sensor_with_meta = sensor_df.merge(meta, on='sensor_id', how='left')


print(meta.head())
print(sensor_df.head())
print()
print(sensor_with_meta.head())

agg_named = sensor_with_meta.groupby(['location']).agg(
temp_mean=('temperature', 'mean'),
temp_med=('temperature', 'median'),
temp_iqr=('temperature', lambda x: np.subtract(*np.percentile(x.dropna(), [75, 25]))),
humidity_mean=('humidity', 'mean'),
n_readings=('temperature', 'count')
)
print('\n--- agg_named by location ---')
print(agg_named)

# custom aggregation using apply (slower but expressive) -> compute time-of-day sensitivity

def day_night_diff(group):
    group = group.copy()
    # daytime mean vs nighttime mean
    group['hour'] = group['timestamp'].dt.hour
    daytime = group.loc[(group['hour'] >= 6) & (group['hour'] < 18), 'temperature']
    nighttime = group.loc[(group['hour'] < 6) | (group['hour'] >= 18), 'temperature']
    return pd.Series({'day_minus_night': daytime.mean() - nighttime.mean()})

sens_daynight = sensor_df.merge(meta, on='sensor_id').groupby('sensor_id').apply(day_night_diff, include_groups=False)
print('\n--- day_minus_night per sensor ---')
print(sens_daynight.head())

  sensor_id location          type
0        S1   Room A  thermo-hygro
1        S2   Room B  thermo-hygro
2        S3   Room C        thermo
3        S4   Room A  thermo-hygro
4        S5   Room D         hygro
  sensor_id           timestamp  temperature   humidity
0        S1 2025-11-01 00:00:00    11.844233  55.829533
1        S1 2025-11-01 00:10:00    13.162956  55.409148
2        S1 2025-11-01 00:20:00    10.544302  64.612578
3        S1 2025-11-01 00:30:00          NaN  63.473544
4        S1 2025-11-01 00:40:00    13.332673  63.221174

  sensor_id           timestamp  temperature   humidity location          type
0        S1 2025-11-01 00:00:00    11.844233  55.829533   Room A  thermo-hygro
1        S1 2025-11-01 00:10:00    13.162956  55.409148   Room A  thermo-hygro
2        S1 2025-11-01 00:20:00    10.544302  64.612578   Room A  thermo-hygro
3        S1 2025-11-01 00:30:00          NaN  63.473544   Room A  thermo-hygro
4        S1 2025-11-01 00:40:00    13.332673  63.221174   

In [7]:
# Note on merge vs join
# pandas.merge() is the core function for all merges/joins.
# df.merge() and df.join() are convenient wrappers:
# - df.join() defaults to left join on df1's index and df2's index.
# - df.merge() defaults to inner join on columns, but can join on indexes.
# Use merge() for flexibility; join() can save typing in simple index-based left joins.

In [8]:
# -------------------------------
# 5) pivot_table & crosstab
# -------------------------------
# pivot_table: average temperature per sensor per hour of day
sensor_df['hour'] = sensor_df['timestamp'].dt.hour
pv = pd.pivot_table(sensor_df, values='temperature', index='sensor_id', columns='hour', aggfunc='mean')
print('\n--- pivot_table mean temperature by sensor x hour ---')
print(pv.iloc[:, :6]) # show first 6 hours as sample

# crosstab: count of readings per sensor per type
sensor_meta_joined = sensor_df.merge(meta, on='sensor_id')
ct = pd.crosstab(sensor_meta_joined['sensor_id'], sensor_meta_joined['type'])
print('\n--- crosstab counts sensor x type ---')
print(sensor_meta_joined.head())
print(ct)


--- pivot_table mean temperature by sensor x hour ---
hour               0          1          2          3          4          5
sensor_id                                                                  
S1         12.210048  10.664322  10.720927  12.989877  14.358660  14.856285
S2          9.690962  10.132154  10.812555  12.572447  13.260633  14.696669
S3         10.254658  10.215027  10.674089  13.038813  12.860999  15.427306
S4          9.326482  10.789089  11.722838  13.088461  13.111283  14.757508
S5         12.241501  12.651350  11.540636  13.750657  14.472117  16.459888
S6         13.411637  12.972633  14.169101  15.001611  16.085027  17.580157

--- crosstab counts sensor x type ---
  sensor_id           timestamp  temperature   humidity  hour location  \
0        S1 2025-11-01 00:00:00    11.844233  55.829533     0   Room A   
1        S1 2025-11-01 00:10:00    13.162956  55.409148     0   Room A   
2        S1 2025-11-01 00:20:00    10.544302  64.612578     0   Room A   
3 

In [9]:
# -------------------------------
# 6) Merging and joining examples (all possibilities)
# -------------------------------
# inner merge (only sensors present in both)
m_inner = sensor_df.merge(meta, on='sensor_id', how='inner')
# left merge (keeps all readings)
m_left = sensor_df.merge(meta, on='sensor_id', how='left')
# right and outer
m_right = sensor_df.merge(meta, on='sensor_id', how='right')
m_outer = sensor_df.merge(meta, on='sensor_id', how='outer')



print('\nmerge sizes: inner={}, left={}, right={}, outer={}'.format(
len(m_inner), len(m_left), len(m_right), len(m_outer)
))

# join on index: prepare two DataFrames with indexes
df_a = sensor_df.sample(50, random_state=4).set_index(['sensor_id', 'timestamp']).sort_index()
df_b = pd.DataFrame({'battery': np.random.randint(20, 100, size=30)},
index=df_a.index[:30])
# left join using index
joined = df_a.join(df_b, how='left')

print('\n--- joined (index join) sample ---')
print(joined.head(), joined.shape)

# concat: stack vertically and horizontally
concat_v = pd.concat([sensor_df.head(5), sensor_df.tail(5)], axis=0)
concat_h = pd.concat([sensor_df.head(5).reset_index(drop=True).iloc[:, :3],
sensor_df.head(5).reset_index(drop=True).iloc[:, 3:]], axis=1)

# combine_first: combine two sources preferring left's non-null values
left = sensor_df.head(10).copy()
right = left.copy()
right.loc[right.sample(frac=0.3, random_state=5).index, 'temperature'] = np.nan
combined = right.combine_first(left)
print('\n--- combine_first example ---')
print(combined.head(20))



merge sizes: inner=864, left=864, right=864, outer=864

--- joined (index join) sample ---
                               temperature   humidity  hour  battery
sensor_id timestamp                                                 
S1        2025-11-01 07:50:00    16.998632  51.400346     7     74.0
          2025-11-01 10:30:00    24.450338  75.506305    10     35.0
          2025-11-01 11:00:00    23.210710  60.886517    11     81.0
          2025-11-01 11:10:00    21.762797  59.342354    11     67.0
          2025-11-01 11:40:00    22.796130  52.161576    11     49.0 (50, 4)

--- combine_first example ---
  sensor_id           timestamp  temperature   humidity  hour
0        S1 2025-11-01 00:00:00    11.844233  55.829533     0
1        S1 2025-11-01 00:10:00    13.162956  55.409148     0
2        S1 2025-11-01 00:20:00    10.544302  64.612578     0
3        S1 2025-11-01 00:30:00          NaN  63.473544     0
4        S1 2025-11-01 00:40:00    13.332673  63.221174     0
5        S1 20

In [10]:
# -------------------------------
# 7) Reshaping: stack/unstack and melt
# -------------------------------
# create a small pivot and show stack/unstack
small = sensor_df[sensor_df['sensor_id'].isin(['S1', 'S2'])].head(12)
pv_small = small.pivot(index='timestamp', columns='sensor_id', values='temperature')
print('\n--- pv_small pivot (timestamp x sensor) ---')
print(pv_small.head())
print('\nstacked -> unstacked roundtrip (demonstration)')
stacked = pv_small.stack()
print(stacked.head())
print(stacked.unstack().head())

# melt: convert wide back to long
melted = pv_small.reset_index().melt(id_vars='timestamp', var_name='sensor_id', value_name='temperature')
print('\n--- melted back to long ---')
print(melted.head())


--- pv_small pivot (timestamp x sensor) ---
sensor_id                   S1
timestamp                     
2025-11-01 00:00:00  11.844233
2025-11-01 00:10:00  13.162956
2025-11-01 00:20:00  10.544302
2025-11-01 00:30:00        NaN
2025-11-01 00:40:00  13.332673

stacked -> unstacked roundtrip (demonstration)
timestamp            sensor_id
2025-11-01 00:00:00  S1           11.844233
2025-11-01 00:10:00  S1           13.162956
2025-11-01 00:20:00  S1           10.544302
2025-11-01 00:40:00  S1           13.332673
2025-11-01 00:50:00  S1           12.166077
dtype: float64
sensor_id                   S1
timestamp                     
2025-11-01 00:00:00  11.844233
2025-11-01 00:10:00  13.162956
2025-11-01 00:20:00  10.544302
2025-11-01 00:40:00  13.332673
2025-11-01 00:50:00  12.166077

--- melted back to long ---
            timestamp sensor_id  temperature
0 2025-11-01 00:00:00        S1    11.844233
1 2025-11-01 00:10:00        S1    13.162956
2 2025-11-01 00:20:00        S1    10.54430

In [11]:
# -------------------------------
# 8) Query: expressive filtering
# -------------------------------
# Use DataFrame.query for readable boolean expressions
q = sensor_df.query('temperature > 25 and humidity < 50')
print(f'\nNumber of readings with temperature>25 and humidity<50: {len(q)}')
print(q.head())

# boolean indexing vs query for more complex expressions
expr = "(temperature > temperature.mean()) & (humidity < humidity.quantile(0.25))"
# careful: can't use temperature.mean() inside query easily; compute first
temp_mean = sensor_df['temperature'].mean()
hum_q25 = sensor_df['humidity'].quantile(0.25)
q2 = sensor_df.query('@temp_mean < temperature and humidity < @hum_q25')
print(f'\nUsing external variables in query, matches: {len(q2)}')


Number of readings with temperature>25 and humidity<50: 12
    sensor_id           timestamp  temperature   humidity  hour
620        S5 2025-11-01 07:20:00    38.594542  42.707012     7
633        S5 2025-11-01 09:30:00    25.573253  42.338233     9
643        S5 2025-11-01 11:10:00    26.768969  38.586943    11
650        S5 2025-11-01 12:20:00    26.089128  49.984683    12
653        S5 2025-11-01 12:50:00    25.895780  43.626175    12

Using external variables in query, matches: 139


In [12]:
# -------------------------------
# 9) Aggregations with groupby + transform and filter
# -------------------------------
# find sensors whose median temperature is > global median
global_median = sensor_df['temperature'].median()
sensor_medians = sensor_df.groupby('sensor_id')['temperature'].median()
hot_sensors = sensor_medians[sensor_medians > global_median]#.index.tolist()
print('\nSensors with median temperature > global median:', hot_sensors)

# use transform to broadcast group statistic back to rows
sensor_df['sensor_temp_median'] = sensor_df.groupby('sensor_id')['temperature'].transform('median')
print(sensor_df.head())
# filter rows where temperature is above group's median
above_group_median = sensor_df[sensor_df['temperature'] > sensor_df['sensor_temp_median']]
print('\nRows above their sensor median (sample):')
print(above_group_median.head())

# groupby.filter to keep only sensors with at least 200 readings
sensors_with_enough = sensor_df.groupby('sensor_id').filter(lambda g: len(g) >= 200)
print('\nSensors kept (len>=200):', sensors_with_enough['sensor_id'].unique())


Sensors with median temperature > global median: sensor_id
S1    17.010690
S5    18.022922
S6    18.631957
Name: temperature, dtype: float64
  sensor_id           timestamp  temperature   humidity  hour  \
0        S1 2025-11-01 00:00:00    11.844233  55.829533     0   
1        S1 2025-11-01 00:10:00    13.162956  55.409148     0   
2        S1 2025-11-01 00:20:00    10.544302  64.612578     0   
3        S1 2025-11-01 00:30:00          NaN  63.473544     0   
4        S1 2025-11-01 00:40:00    13.332673  63.221174     0   

   sensor_temp_median  
0            17.01069  
1            17.01069  
2            17.01069  
3            17.01069  
4            17.01069  

Rows above their sensor median (sample):
   sensor_id           timestamp  temperature   humidity  hour  \
29        S1 2025-11-01 04:50:00    17.846883  64.052320     4   
32        S1 2025-11-01 05:20:00    17.064629  58.211946     5   
37        S1 2025-11-01 06:10:00    17.429709  61.295790     6   
38        S1 2025

In [13]:
# -------------------------------
# 10) MultiIndex columns: produce aggregated table with hierarchical columns and plot
# -------------------------------
agg_multi = sensor_df.groupby('sensor_id').agg(
temp_mean=('temperature', 'mean'),
temp_std=('temperature', 'std'),
hum_mean=('humidity', 'mean'),
hum_std=('humidity', 'std')
)
print(agg_multi)
print('\n\n')
# create MultiIndex columns manually
agg_multi.columns = pd.MultiIndex.from_tuples([('temperature', 'mean'), ('temperature', 'std'), ('humidity', 'mean'), ('humidity', 'std')])
print('\n--- agg_multi (MultiIndex columns) ---')
print(agg_multi)

           temp_mean  temp_std   hum_mean   hum_std
sensor_id                                          
S1         16.718369  4.511676  59.259889  5.560005
S2         15.427998  4.497470  45.599224  5.493384
S3         15.658394  4.421943  56.385015  5.146519
S4         15.668049  4.238744  48.812680  5.493305
S5         17.801163  4.889507  49.913718  5.367881
S6         18.627710  4.633420  51.732933  5.117629




--- agg_multi (MultiIndex columns) ---
          temperature             humidity          
                 mean       std       mean       std
sensor_id                                           
S1          16.718369  4.511676  59.259889  5.560005
S2          15.427998  4.497470  45.599224  5.493384
S3          15.658394  4.421943  56.385015  5.146519
S4          15.668049  4.238744  48.812680  5.493305
S5          17.801163  4.889507  49.913718  5.367881
S6          18.627710  4.633420  51.732933  5.117629


In [14]:
# -------------------------------
# 11) Rolling, expanding and time-based resampling (for sensor S2)
# -------------------------------
s2 = sensor_df[sensor_df['sensor_id'] == 'S2'].set_index('timestamp').sort_index()

# rolling 30-minute window (freq is 10min so 3 observations -> use time-based window)
s2['temp_roll_30min'] = s2['temperature'].rolling('30min').mean()
# humidity diff next reading -> shift(-1)
s2['humidity_next_diff'] = s2['humidity'].shift(-1) - s2['humidity']
# boolean: temp increased >5C and humidity dropped >20 compared to previous measurement
s2['temp_prev_diff'] = s2['temperature'] - s2['temperature'].shift(1)
s2['hum_prev_diff'] = s2['humidity'].shift(1) - s2['humidity']
s2['extreme_change'] = (s2['temp_prev_diff'] > 5) & (s2['hum_prev_diff'] > 20)

## Task Specification: Aggregations, Joins, GroupBy, and Rolling Analysis with Pandas

---

### **Task Overview**
This assignment simulates a **customer purchase analytics workflow** using Pandas. You will work with timestamped transaction data and customer metadata, performing:

- Group-based aggregations
- Time-based analysis
- Rolling window calculations within groups
- Joining customer metadata from another dataset

---

### **Datasets**
You are provided with two datasets:

| File | Description |
|------|-------------|
| `data/purchases.csv` | Timestamped purchase data (contains `Customer_id`, `Timestamp`, `Price`, etc.) |
| `data/customer_info.csv` | Customer metadata (contains `Customer_id`, `Born_date`, `City`, etc.) |

---

### **1. Load and Prepare the Data**
- **Task 1.1:** Load the two CSV files into Pandas DataFrames:
  - Use `df` for `purchases.csv`
  - Use `customer_info` for `customer_info.csv`

- **Task 1.2:** Convert the `Timestamp` column in `df` to Pandas `datetime` format.

- **Task 1.3:** Set `Timestamp` as the **index** of `df` and **sort** the index in ascending order.

---

### **2. Join Customer Metadata**
- **Task 2.1:** Merge the purchase DataFrame (`df`) with the customer metadata DataFrame (`customer_info`) on the `Customer_id` column.

- **Task 2.2:** The resulting DataFrame should include the columns `Born_date` and `City` from the `customer_info` DataFrame.

---

### **3. GroupBy and Aggregations**
Using the merged DataFrame:

- **Task 3.1:** Group by **Customer_id** and compute:
  - Total revenue (`sum(Price)`)
  - Number of purchases (`count`)
  - Average purchase price (`mean(Price)`)

- **Task 3.2:** Create a new feature by extracting the **month name** (e.g., `"January"`, `"February"`) from the `Timestamp`.

- **Task 3.3:** Group by **City** and **Month** and compute:
  - Total monthly revenue per city (`sum(Price)`)

---

### **4. Rolling Window Analysis (Per Customer)**
Perform rolling window calculations **within each customer group**:

- Inside each customer group:
  - **Sort** the DataFrame by `Timestamp`.

- Compute the following:
  - A **3-purchase rolling average** of `Price`.
  - A `diff()` column showing the change in price compared to the previous purchase.

> **Note:** The rolling operations must be computed **within each customer group**, not across all customers.

---

### **5. Advanced Filtering**
- Do the same as in 4 only for customers from **Berlin**, make sure only necessary values are joined (that is filter first, join second).

- Do the same as in 4 only for rows where the `Price` of the current purchase is greater than **20%** than the previous purchase (`Price`), make sure only necessary values are joined (that is filter first, join second).

---

### **6. Custom Lambda Function for GroupBy Operations**
- Apply a **custom lambda function** within a `groupby()` operation to filter or modify the data. For example:
  - Use a lambda to **filter** customers who have purchased more than a specific amount within a given timeframe (e.g., greater than $1000 in total purchases during the last quarter).
  - Apply custom transformations to grouped data using the lambda function.

---

### 7. Time-Based Analysis
- Perform additional time-based analysis:
  - Calculate **monthly growth rate** for total revenue per customer (i.e., compare total revenue of the current month with the previous month).
  - Create a **cumulative sum** of purchases over time per customer.


In [65]:
import pandas as pd
import numpy as np



#1

#load data
df = pd.read_csv('data/purchases.csv')
customer_info = pd.read_csv('data/customer_info.csv')

#time format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

#set time as index
df = df.set_index('Timestamp').sort_index()




#2

#merge 2 df
df_merged = df.merge(customer_info, on='Customer_id')





#3
#df with agg functions, indices are customer id
agg_named = df.groupby(['Customer_id']).agg(Total_revenue=('Price', 'sum'), Number_of_Purchases=('Price', 'count'), Average_Purchase_Price=('Price', 'mean'))

#merge with the df, reset index
df_merged = df_merged.merge(agg_named, on='Customer_id')

df_merged.index = df.index  #set timestamp as index
df = df_merged

df['Month'] = df.index.month
df['Month_name'] = df.index.month_name()

#total monthly revenue as column in df
df['Total_Monthly_Revenue'] = df.groupby(['City', 'Month'])['Price'].transform('sum')
#total monthly revenue as pivot table
pv = pd.pivot_table(df, values='Price', index='City', columns='Month', aggfunc='sum')
display(pv)






#4.

#sort each group by index
df = df.groupby(['Customer_id']).apply(lambda g: g.sort_index(level = 'Timestamp'), include_groups=False)#.reset_index(level=0)

#rolling average for each group - creates multiindex => need to reset index
df['3_Rolling_Average_Price'] = df.groupby(['Customer_id'])['Price'].rolling(3).mean().reset_index(level=0, drop = True)

#shifted
df['Price_Previous_Purchase'] = df.groupby(['Customer_id'])['Price'].shift(1)
df['Difference_from_Last_Purchase'] = df['Price'] - df['Price_Previous_Purchase']






#5


#filter berlin customers
df_new = df[df['City'] == 'Berlin']

#sort each group by timestamp
df_new = df_new.groupby(['Customer_id']).apply(lambda g: g.sort_index(level = 'Timestamp'), include_groups=False).reset_index(level=0, drop = True)

#rolling average for berlin
df_new['Berlin_3_Rolling_Average_Price'] = df_new.groupby(['Customer_id'])['Price'].rolling(3).mean().reset_index(level=0, drop = True)

#difference for berlin
df_new['Berlin_Difference_from_Last_Purchase'] = df_new['Price'] - df_new.groupby(['Customer_id'])['Price'].shift(1)

#join
df = df.join(df_new['Berlin_Difference_from_Last_Purchase'], how='outer').join(df_new['Berlin_3_Rolling_Average_Price'])






#filter difference > 20%
df_new = df[df['Difference_from_Last_Purchase'] / df['Price_Previous_Purchase'] > 0.2 ]

#sort
df_new = df_new.groupby(['Customer_id']).apply(lambda g: g.sort_index(level = 'Timestamp'), include_groups=False).reset_index(level=0, drop = True)

#rolling average
df_new['20%_3_Rolling_Average_Price'] = df_new.groupby(['Customer_id'])['Price'].rolling(3).mean().reset_index(level=0, drop = True)

#difference
df_new['20%_Difference_from_Last_Purchase'] = df_new['Price'] - df_new.groupby(['Customer_id'])['Price'].shift(1)

#join
df = df.join(df_new['20%_Difference_from_Last_Purchase'], how='outer').join(df_new['20%_3_Rolling_Average_Price'])






#filter purchases in the first 5 days from 01.06.
mask = df.index.get_level_values('Timestamp') < '2025-01-06 00:00:00'

#column: purchases in the first 5 days -> sum, other -> Nan
df['Sum_First_5_Days'] = df[mask].groupby(['Customer_id'])['Price'].transform('sum')

#df, where all customers with sum in the first 5 days are filtered
df_sum_in_first_5_days_bigger_than_1000 = df.groupby(['Customer_id']).filter(lambda g: g['Sum_First_5_Days'].max() > 1000)







#6


#column with cumsum
df['Cumsum'] = df.groupby(['Customer_id'])['Price'].cumsum()


#column with monthly sum of prices of purchases
df['Monthly_sum'] = df.groupby(['Customer_id', 'Month'])['Price'].transform('sum')

#pivot table with monthly growth rate
pv = pd.pivot_table(df, values='Price', index='Customer_id', columns='Month', aggfunc='sum')
pv['Second_Month_Growth_Rate'] = pv[2] / pv[1]
pv['Third_Month_Growth_Rate'] = pv[3] / pv[2]
display(pv)


display(df)

Month,1,2,3
City,,,
Berlin,101192.17,86163.85,10474.86
Cologne,98030.00,86781.31,11220.64
Frankfurt,61492.29,52020.01,6698.88
Hamburg,43988.43,42056.17,5340.12
Munich,78726.19,67799.84,7330.35


Month,1,2,3,Second_Month_Growth_Rate,Third_Month_Growth_Rate
Customer_id,,,,,
C001,7302.10,5204.79,548.86,0.712780,0.105453
C002,7538.37,7838.50,550.93,1.039814,0.070285
C003,7133.09,7141.33,672.92,1.001155,0.094229
C004,6920.10,7008.12,1183.21,1.012719,0.168834
C005,7928.47,7284.34,1585.80,0.918757,0.217700
C006,8113.09,5486.55,1174.60,0.676259,0.214087
C007,6739.05,4565.22,581.80,0.677428,0.127442
C008,6725.07,8830.59,NaN,1.313085,NaN
C009,7291.73,5082.35,256.29,0.697002,0.050427


Price   Born_date       City  Total_revenue  \
Customer_id Timestamp                                                           
C001        2025-01-01 05:30:00  376.61  1998-12-03  Frankfurt       13055.75   
            2025-01-02 02:00:00   63.23  1998-12-03  Frankfurt       13055.75   
            2025-01-02 10:00:00  108.83  1998-12-03  Frankfurt       13055.75   
            2025-01-05 05:00:00   70.19  1998-12-03  Frankfurt       13055.75   
            2025-01-06 07:00:00  132.80  1998-12-03  Frankfurt       13055.75   
...                                 ...         ...        ...            ...   
C050        2025-02-26 06:30:00  269.66  1987-08-04     Berlin       12643.75   
            2025-03-01 07:30:00  154.22  1987-08-04     Berlin       12643.75   
            2025-03-01 22:00:00  492.96  1987-08-04     Berlin       12643.75   
            2025-03-02 20:00:00  248.72  1987-08-04     Berlin       12643.75   
            2025-03-03 13:30:00  136.22  1987-08-04     Berlin       12643.75   

                                 Number_of_Purchases  Average_Purchase_Price  \
Customer_id Timestamp                                                          
C001        2025-01-01 05:30:00                   56              233.138393   
            2025-01-02 02:00:00                   56              233.138393   
            2025-01-02 10:00:00                   56              233.138393   
            2025-01-05 05:00:00                   56              233.138393   
            2025-01-06 07:00:00                   56              233.138393   
...                                              ...                     ...   
C050        2025-02-26 06:30:00                   59              214.300847   
            2025-03-01 07:30:00                   59              214.300847   
            2025-03-01 22:00:00                   59              214.300847   
            2025-03-02 20:00:00                   59              214.300847   
            2025-03-03 13:30:00                   59              214.300847   

                                 Month Month_name  Total_Monthly_Revenue  \
Customer_id Timestamp                                                      
C001        2025-01-01 05:30:00      1    January               61492.29   
            2025-01-02 02:00:00      1    January               61492.29   
            2025-01-02 10:00:00      1    January               61492.29   
            2025-01-05 05:00:00      1    January               61492.29   
            2025-01-06 07:00:00      1    January               61492.29   
...                                ...        ...                    ...   
C050        2025-02-26 06:30:00      2   February               86163.85   
            2025-03-01 07:30:00      3      March               10474.86   
            2025-03-01 22:00:00      3      March               10474.86   
            2025-03-02 20:00:00      3      March               10474.86   
            2025-03-03 13:30:00      3      March               10474.86   

                                 3_Rolling_Average_Price  \
Customer_id Timestamp                                      
C001        2025-01-01 05:30:00                      NaN   
            2025-01-02 02:00:00                      NaN   
            2025-01-02 10:00:00               182.890000   
            2025-01-05 05:00:00                80.750000   
            2025-01-06 07:00:00               103.940000   
...                                                  ...   
C050        2025-02-26 06:30:00               304.080000   
            2025-03-01 07:30:00               285.203333   
            2025-03-01 22:00:00               305.613333   
            2025-03-02 20:00:00               298.633333   
            2025-03-03 13:30:00               292.633333   

                                 Price_Previous_Purchase  \
Customer_id Timestamp                                      
C001        2025-01-01 05:30:00                  